In [2]:
from cdapython import Q, columns, unique_terms

In [2]:
columns()

SELECT field_path FROM `gdc-bq-sample.cda_mvp.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS` WHERE table_name = 'v3'


['days_to_birth',
 'race',
 'sex',
 'ethnicity',
 'id',
 'ResearchSubject',
 'ResearchSubject.Diagnosis',
 'ResearchSubject.Diagnosis.morphology',
 'ResearchSubject.Diagnosis.tumor_stage',
 'ResearchSubject.Diagnosis.tumor_grade',
 'ResearchSubject.Diagnosis.Treatment',
 'ResearchSubject.Diagnosis.Treatment.type',
 'ResearchSubject.Diagnosis.Treatment.outcome',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.primary_diagnosis',
 'ResearchSubject.Diagnosis.age_at_diagnosis',
 'ResearchSubject.Specimen',
 'ResearchSubject.Specimen.File',
 'ResearchSubject.Specimen.File.label',
 'ResearchSubject.Specimen.File.associated_project',
 'ResearchSubject.Specimen.File.drs_uri',
 'ResearchSubject.Specimen.File.identifier',
 'ResearchSubject.Specimen.File.identifier.system',
 'ResearchSubject.Specimen.File.identifier.value',
 'ResearchSubject.Specimen.File.data_category',
 'ResearchSubject.Specimen.File.byte_size',
 'ResearchSubject.Specimen.File.type',
 'ResearchSubject.Specimen.File

In [3]:
unique_terms('ResearchSubject.associated_project')

SELECT DISTINCT(_ResearchSubject.associated_project) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject ORDER BY _ResearchSubject.associated_project


['Academia Sinica LUAD-100',
 'BEATAML1.0-COHORT',
 'BEATAML1.0-CRENOLANIB',
 'CGCI-BLGSP',
 'CGCI-HTMCP-CC',
 'CMI-ASC',
 'CMI-MBC',
 'CMI-MPC',
 'CPTAC-2',
 'CPTAC-3',
 'CPTAC-TCGA',
 'CPTAC2 Retrospective',
 'CPTAC3-Discovery',
 'CTSP-DLBCL1',
 'FM-AD',
 'GENIE-DFCI',
 'GENIE-GRCC',
 'GENIE-JHU',
 'GENIE-MDA',
 'GENIE-MSK',
 'GENIE-NKI',
 'GENIE-UHN',
 'GENIE-VICC',
 'Georgetown Lung Cancer Proteomics Study',
 'HCMI-CMDC',
 'Human Early-Onset Gastric Cancer - Korea University',
 'Integrated Proteogenomic Characterization of HBV-related Hepatocellular carcinoma',
 'MMRF-COMMPASS',
 'NCICCR-DLBCL',
 'OHSU-CNL',
 'ORGANOID-PANCREATIC',
 'Oral Squamous Cell Carcinoma - Chang Gung University',
 'PJ25730263',
 'Proteogenomic Analysis of Pediatric Brain Cancer Tumors Pilot Study',
 'TARGET-ALL-P1',
 'TARGET-ALL-P2',
 'TARGET-ALL-P3',
 'TARGET-AML',
 'TARGET-CCSK',
 'TARGET-NBL',
 'TARGET-OS',
 'TARGET-RT',
 'TARGET-WT',
 'TCGA-ACC',
 'TCGA-BLCA',
 'TCGA-BRCA',
 'TCGA-CESC',
 'TCGA-CHOL',
 

<font color=blue> The following example is from the CDA testing and I've put it here to show how to use UNNEST to get down to returing specific fields.  It looks like there needs to be a serial unnesting</font>

In [4]:
unique_terms("ResearchSubject.Specimen.source_material_type", system="PDC")

SELECT DISTINCT(_Specimen.source_material_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen,UNNEST(_ResearchSubject.identifier) AS _identifier WHERE _identifier.system="PDC" ORDER BY _Specimen.source_material_type


['Cell Lines',
 'Normal',
 'Normal Adjacent Tissue',
 'Not Reported',
 'Primary Tumor',
 'Solid Tissue Normal',
 'Tumor',
 'Xenograft',
 'Xenograft Tissue']

In [4]:
q1 = Q('ResearchSubject.Diagnosis.age_at_diagnosis > 50*365')
q2 = Q('ResearchSubject.associated_project = "TCGA-OV"')

q = q1.And(q2)
r = q.run()

print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_Diagnosis.age_at_diagnosis > 50*365) AND (_ResearchSubject.associated_project = 'TCGA-OV'))
Offset: 0
Limit: 1000
Count: 461
More pages: No



In [5]:
q1 = Q('ResearchSubject.Specimen.primary_disease_type = "Nevi and Melanomas"')
q2 = Q('ResearchSubject.Diagnosis.age_at_diagnosis < 30*365')

q = q1.And(q2)
r = q.run()

print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_Specimen.primary_disease_type = 'Nevi and Melanomas') AND (_Diagnosis.age_at_diagnosis < 30*365))
Offset: 0
Limit: 1000
Count: 647
More pages: No



In [6]:
dir(r)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api_response',
 '_get_result',
 '_host',
 '_limit',
 '_offset',
 '_version',
 'count',
 'next_page',
 'pretty_print',
 'prev_page',
 'sql']

In [10]:
r.count

647

In [9]:
tumor_type = Q('ResearchSubject.Specimen.source_material_type = "Primary Tumor"')
disease1 = Q('ResearchSubject.primary_disease_site = "Ovary"')
disease2 = Q('ResearchSubject.primary_disease_site = "Breast"')
demographics1 = Q('sex = "female"')
demographics2 = Q('days_to_birth > -60*365') # note that days_to_birth is a negative value

q1 = tumor_type.And(demographics1.And(demographics2))
q2 = disease1.Or(disease2)
q = q1.And(q2)

r = q.run()
print(r)




Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (((_Specimen.source_material_type = 'Primary Tumor') AND ((v3.sex = 'female') AND (v3.days_to_birth > -60*365))) AND ((_ResearchSubject.primary_disease_site = 'Ovary') OR (_ResearchSubject.primary_disease_site = 'Breast')))
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes



In [10]:
limit = 1000
loopcounter = 1
casecounter = 0
while r.count == limit:
    print(("Loop Count: %s\tReturn Count: %s\tCase Count: %s\n") % (str(loopcounter), str(r.count), str(casecounter)))
    loopcounter = loopcounter + 1
    casecounter = casecounter + r.count
    r = r.next_page()

Loop Count: 1	Return Count: 1000	Case Count: 0

Loop Count: 2	Return Count: 1000	Case Count: 1000

Loop Count: 3	Return Count: 1000	Case Count: 2000

Loop Count: 4	Return Count: 1000	Case Count: 3000

Loop Count: 5	Return Count: 1000	Case Count: 4000

Loop Count: 6	Return Count: 1000	Case Count: 5000

Loop Count: 7	Return Count: 1000	Case Count: 6000

Loop Count: 8	Return Count: 1000	Case Count: 7000

Loop Count: 9	Return Count: 1000	Case Count: 8000

Loop Count: 10	Return Count: 1000	Case Count: 9000

Loop Count: 11	Return Count: 1000	Case Count: 10000

Loop Count: 12	Return Count: 1000	Case Count: 11000

Loop Count: 13	Return Count: 1000	Case Count: 12000

Loop Count: 14	Return Count: 1000	Case Count: 13000

Loop Count: 15	Return Count: 1000	Case Count: 14000

Loop Count: 16	Return Count: 1000	Case Count: 15000

Loop Count: 17	Return Count: 1000	Case Count: 16000

Loop Count: 18	Return Count: 1000	Case Count: 17000

Loop Count: 19	Return Count: 1000	Case Count: 18000

Loop Count: 20	

In [3]:
unique_terms("ResearchSubject.Specimen.File.data_category")

SELECT DISTINCT(_File.data_category) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen,UNNEST(_Specimen.File) AS _File ORDER BY _File.data_category


['Biospecimen',
 'Combined Nucleotide Variation',
 'Copy Number Variation',
 'DNA Methylation',
 'Peptide Spectral Matches',
 'Processed Mass Spectra',
 'Raw Mass Spectra',
 'Sequencing Reads',
 'Simple Nucleotide Variation',
 'Somatic Structural Variation',
 'Structural Variation',
 'Transcriptome Profiling']